Если у вас еще нет интерфейса командной строки Yandex Cloud, установите и инициализируйте его: https://cloud.yandex.ru/docs/cli/quickstart#install

Получите IAM-токен:

In [ ]:
!yc iam create-token

In [ ]:
# Необходимо указать полученный IAM token
iam_token = ""

In [ ]:
import requests

In [ ]:
# Список всех доступных организаций
res = requests.get('https://resource-manager.api.cloud.yandex.net/resource-manager/v1/clouds', 
                    headers={"Authorization" : "Bearer {}".format(iam_token)})

res.json()

In [ ]:
# Задаем organizationId организации, в которой должно быть создано сообщество
ORGANIZATION_ID = ""

In [ ]:
# Получаем список доступных биллинг аккаунтов:
res = requests.get('https://billing.api.cloud.yandex.net/billing/v1/billingAccounts', 
                    headers={"Authorization" : "Bearer {}".format(iam_token)})
res.json()

In [ ]:
# Создаем новое сообщество 

data={}
data['name'] = "Automatically created community 1"
data['description'] = "This is a community for our first project"
data['organizationId'] = ORGANIZATION_ID
data['billingAccountId'] = "" #id активного биллинг аккаунта

res = requests.post('https://datasphere.api.cloud.yandex.net/datasphere/v2/communities', 
                    json=data,
                    headers={"Authorization" : "Bearer {}".format(iam_token)})
community_res = res.json()
community_res

In [ ]:
# Cписок ID ролей в DataSphere
res = requests.get('https://iam.api.cloud.yandex.net/iam/v1/roles', 
                    headers={"Authorization" : "Bearer {}".format(iam_token)})
roles = res.json()['roles']
datasphere_roles = [role for role in roles if 'datasphere' in role['id']]
datasphere_roles

In [ ]:
# Список всех участников организации
res = requests.get("https://organization-manager.api.cloud.yandex.net/organization-manager/v1/organizations/{}/users".format(ORGANIZATION_ID), 
                    headers={"Authorization" : "Bearer {}".format(iam_token)})
res.json()

In [ ]:
# Создаем проекты

# Указываем список id участников (поле 'sub'), для которых должны быть созданы проекты
user_organization_ids = ['ajesma7imirhrg1m0hgo']

projects = {}
for user_id in user_organization_ids:
    # Создаем проект для пользователя
    data={}
    data['name'] = "Student {}".format(user_id)
    data['communityId'] = community_res['metadata']['communityId']
    data['description'] = "This is a workplace and create code and store resources"
    data['limits'] = {
        # Задаем ограничение на максимальное количество юнитов в час для проекта
        "maxUnitsPerHour": 10000,
        # Задаем ограничение на максимальное количество юнитов на один запуск ячейки для проекта
        "maxUnitsPerExecution": 5000
    }
    res = requests.post('https://datasphere.api.cloud.yandex.net/datasphere/v2/projects', 
                        json=data,
                        headers={"Authorization" : "Bearer {}".format(iam_token)})
    print("Project for {} is created with response: {}".format(user_id, res))
    
    data={}
    data['communityId'] = community_res['metadata']['communityId']
    data['projectNamePattern'] = "Student {}".format(user_id)
    res = requests.get('https://datasphere.api.cloud.yandex.net/datasphere/v2/projects', 
                        json=data,
                        headers={"Authorization" : "Bearer {}".format(iam_token)})
    projects[user_id] = res.json()['projects'][0]
    project_id = res.json()['projects'][0]['id']
    
    # Добавляем себя в проект админом и пользователя с ролью developer
    data={}
    data['accessBindings'] = [{
        "roleId": 'datasphere.community-projects.admin',
        "subject": {
            "id": "", # Необходимо указать ID администратора проекта
            "type": "userAccount"
        }},
        {
        "roleId": 'datasphere.community-projects.developer',
        "subject": {
            "id": user_id,
            "type": "userAccount"    
        }}
    ]
    res = requests.post('https://datasphere.api.cloud.yandex.net/datasphere/v2/projects/{}:setAccessBindings'.\
                        format(project_id), 
                        json=data,
                        headers={"Authorization" : "Bearer {}".format(iam_token)})
    print("Admin was added to project {} with response: {}".format(project_id, res))

In [ ]:
projects